In [1]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torch import nn
from PIL import Image
import pandas as pd
import os
import torchvision.io
from torchvision.transforms import ToTensor
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
NUM_WORKERS = 0

print(f"Using {DEVICE} device")

Using cuda device


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        image = read_image(img_path,ImageReadMode.RGB).float()
       
        label = self.img_labels.iloc[idx, 4]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

class Data:
    def __init__(self, batch_size,dataset_train,dataset_test):
        self.batch_size = batch_size
        self.training_data = dataset_train
        self.test_data = dataset_test
    
    def get_loader(self, training: bool):
        if training:
            dataloader = DataLoader(self.training_data,batch_size=self.batch_size, shuffle=True)
        else:
            dataloader = DataLoader(self.test_data,batch_size=self.batch_size, shuffle=False)
        return dataloader

In [ ]:
class Data:
    def __init__(self, batch_size,dataset_train,dataset_test):
        self.batch_size = batch_size
        self.training_data = dataset_train
        self.test_data = dataset_test
    
    def get_loader(self, training: bool):
        if training:
            dataloader = DataLoader(self.training_data,batch_size=self.batch_size, shuffle=True)
        else:
            dataloader = DataLoader(self.test_data,batch_size=self.batch_size, shuffle=False)
        return dataloader

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 12, kernel_size=7, stride=1, padding=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(12 * 112*112, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,15)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.flatten(x)
        
        x = self.linear_relu_stack(x)
        return x

In [ ]:
class Learner:
    def __init__(self):
        self.model = NeuralNetwork()
        self.model.to(DEVICE)
        self.optimizer =  torch.optim.SGD(self.model.parameters(), lr=0.5e-7)
    
    def predict(self, x):
        return self.model(x)
    
    def update(self, loss):
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
class Evaluator:
    def __init__(self):
        self.loss_fn = nn.CrossEntropyLoss()
            
    def get_loss(self, y, y_hat):
        return self.loss_fn(y_hat,y)

In [ ]:
class Trainer:
    def __init__(self, data: Data, learner: Learner, evaluator: Evaluator):
        self.data = data
        self.learner = learner
        self.evaluator = evaluator
        self.writer = SummaryWriter()
    def one_epoch(self, training: bool):
        self.learner.model.train(training)
        dataloader = self.data.get_loader(training)
        test_loss,train_loss, correct = 0, 0, 0
        predictions = []
        labels = []
        num_batches = len(dataloader)
        size = len(dataloader.dataset)
        for batch_idx, (X, y) in enumerate(dataloader):
            X, y = X.to(DEVICE), y.to(DEVICE)
            y_hat = self.learner.predict(X)
            loss = self.evaluator.get_loss(y, y_hat)
            if training:
                self.learner.update(loss)
                train_loss+=loss.item()
                if batch_idx % 500 == 0:
                    loss, current = loss.item(), (batch_idx + 1) * len(X)
                    print(loss)
                    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            else:
                test_loss += loss.item()
                predictions.extend(y_hat.argmax(1).tolist()) 
                correct += (y_hat.argmax(1) == y).sum().item()
                labels.extend(y.tolist())
                                
        if not training:
            test_loss /= num_batches
            correct /= size
            print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            accuracy = correct*100
            return test_loss, accuracy, predictions,labels
        else:
            train_loss /= num_batches
            return train_loss
    
    def run(self, n_epochs: int):
        for t in range(n_epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            train_loss = self.one_epoch(training=True)
            with torch.no_grad():
                test_loss,test_acc, preds, labels= self.one_epoch(training=False)               
                test_f1 = f1_score(labels,preds, average='macro')*100
                
                
            self.writer.add_scalar('Loss/Train',train_loss, t)
            self.writer.add_scalar('Loss/Test',test_loss, t)
            self.writer.add_scalar('Loss/Test-acc',test_acc, t)
            self.writer.add_scalar('Loss/Test-f1',test_f1, t)
            
            
            
        print("Done!")

In [ ]:
train = 'train'
test = 'test'
path_train = f'dataset/spiders_{train}.csv'
path_test = f'dataset/spiders_{test}.csv'
base_dir = 'dataset'
dataset_train = CustomImageDataset(path_train,base_dir)
dataset_test = CustomImageDataset(path_test,base_dir)

In [ ]:
data = Data(8, dataset_train,dataset_test)
learner = Learner()
evaluator = Evaluator()
trainer = Trainer(data, learner, evaluator)

In [ ]:
trainer.run(100)

Epoch 1
-------------------------------
5.9513373374938965
loss: 5.951337  [    8/ 2185]
Test Error: 
 Accuracy: 6.7%, Avg loss: 3.556385 

Epoch 2
-------------------------------
3.8347630500793457
loss: 3.834763  [    8/ 2185]
Test Error: 
 Accuracy: 8.0%, Avg loss: 3.356377 

Epoch 3
-------------------------------
3.5299854278564453
loss: 3.529985  [    8/ 2185]
Test Error: 
 Accuracy: 9.3%, Avg loss: 3.254321 

Epoch 4
-------------------------------
2.7150661945343018
loss: 2.715066  [    8/ 2185]
Test Error: 
 Accuracy: 10.7%, Avg loss: 3.168898 

Epoch 5
-------------------------------
3.061831474304199
loss: 3.061831  [    8/ 2185]
Test Error: 
 Accuracy: 13.3%, Avg loss: 3.099232 

Epoch 6
-------------------------------
2.452343702316284
loss: 2.452344  [    8/ 2185]
Test Error: 
 Accuracy: 12.0%, Avg loss: 3.047474 

Epoch 7
-------------------------------
2.7932071685791016
loss: 2.793207  [    8/ 2185]
Test Error: 
 Accuracy: 13.3%, Avg loss: 2.982617 

Epoch 8
----------